### States hierarchy

Import the 52 states in 2 differents streams

In [ ]:
!import.sh -o consumertree -l import/states-hierarchy -m 

### Customers

Import the 89,997,827 Customers/IDCards in 2 differents streams

In [ ]:
!csvImport.sh -t 10 -p 16 -o ConsumerProducers -serverThreads 32 -b 50000 \
  -f ../../data/sources/all-id-cards.csv -m -l import/customers

### Accounts

Import the 179,998,862 Accouts/Correspondence in 2 differents streams

In [ ]:
!csvImport.sh -t 10 -p 16 -o AccountProducers -serverThreads 32 -b 50000 \
  -f ../../data/sources/all-accounts.csv -m -l import/accounts

### Non Archived Statememnts

Generate 179,998,862*6 statements in 2 different streams.

Because of a bug in the data generation lib, we need to generate with a month offset of 6 to actually get the last 6 months.

In [23]:
ns = 179998862* 6
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 1079993172
expected duration (s) = 26999.8293


In [ ]:
!import.sh -o statements -l import/statements_live -m  -d 6 -monthOffset 6  \
  -smartPartitioning -n 1079993172 -t 240 -p 24 -a -w 30000

### Achived Statements - part 0

This time, we generate in single repository and without offset to get the first 6 months of the year.

In [22]:
ns = 179998862* 6
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 1079993172
expected duration (s) = 26999.8293


In [ ]:
!import.sh -o statements -l import/statements_archive0 -d 6 -monthOffset 0  \
  -smartPartitioning -n 1079993172 -t 240 -p 24 -a -w 30000

### Achived Statements - part 1

Generate Statements 1
year at a time in single repository mode.

In [24]:
ns = 179998862* 12
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 2159986344
expected duration (s) = 53999.6586


In [ ]:
!import.sh -o statements -l import/statements_archive1 -d 12 -monthOffset 12  \
  -smartPartitioning -n 2159986344 -t 240 -p 24 -a -w 65000

### Achived Statements - part 2

1 year

In [ ]:
!import.sh -o statements -l import/statements_archive2 -d 12 -monthOffset 24  \
  -smartPartitioning -n 2159986344 -t 240 -p 24 -a -w 65000

### Achived Statements - part 3

In [ ]:
!import.sh -o statements -l import/statements_archive3 -d 12 -monthOffset 36  \
  -smartPartitioning -n 2159986344 -t 240 -p 24 -a -w 65000

### Achived Statements - part 4

In [ ]:
!import.sh -o statements -l import/statements_archive4 -d 12 -monthOffset 48  \
  -smartPartitioning -n 2159986344 -t 240 -p 24 -a -w 65000